In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#import wrangle
import env
import seaborn as sns
import acquire
import prep

In [4]:
#single family residence from zillow DB
df = prep.prep_df()

In [5]:
df.isnull().sum().sort_values(ascending=False)

regionidcity                  1008
regionidzip                     64
yearbuilt                       27
logerror                         0
assessmentyear                   0
structuretaxvaluedollarcnt       0
regionidcounty                   0
propertycountylandusecode        0
lotsizesquarefeet                0
longitude                        0
latitude                         0
fips                             0
tax_value                        0
sqft                             0
bedrooms                         0
bathrooms                        0
dtype: int64

In [ ]:
#drops columns with < 80% of values filled in
df = prep.impute_values(df)
df = prep.drop_columns(df)

In [ ]:
df.head()

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
df[['landtaxvaluedollarcnt','lotsizesquarefeet']].head()

In [ ]:
df['tax_value_per_foot'] = df.landtaxvaluedollarcnt/df.lotsizesquarefeet
df.tax_value_per_foot.mean()

In [ ]:
(df.structuretaxvaluedollarcnt/df.tax_value).mean()

In [ ]:
df.lotsizesquarefeet.head()

In [ ]:
df.landtaxvaluedollarcnt /df.tax_value_per_foot.mean()

In [ ]:
df.isnull().sum().sort_values(ascending=False)